# Phase Coherence Analysis with K-means Clustering

This notebook performs phase coherence analysis on fMRI BOLD signals across different consciousness states (Awake, Deep Sleep, and Recovery). The analysis involves:

1. Loading and preprocessing BOLD time series data
2. Computing phase synchronization using Hilbert transform
3. Analyzing phase patterns using k-means clustering
4. Visualizing results and comparing states

## Requirements
- NumPy: For numerical computations
- SciPy: For signal processing and scientific computing
- Pandas: For data manipulation
- Matplotlib: For visualization
- Seaborn: For enhanced plotting
- Scikit-learn: For k-means clustering

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import scipy
import scipy.signal as spsg
from scipy import stats
from scipy.io import loadmat
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.spatial.distance import cityblock

# Set random seed for reproducibility
np.random.seed(42)

# Set plotting style
plt.style.use('seaborn')
sns.set_context('talk')

## Data Loading and Preprocessing

In this section, we load BOLD time series data from three different consciousness states:
1. Awake: Normal conscious state
2. Deep: Deep sleep state
3. Recovery: Post-sleep recovery state

The data is organized using the DK68 atlas, which divides the brain into 68 regions. Each region has its own time series of BOLD signals.


In [ ]:
def load_bold_data(data_path, state):
    """Load BOLD time series data for a given consciousness state.
    
    Args:
        data_path (str): Base path to data directory
        state (str): Consciousness state ('Awake', 'Deep', or 'Recovery')
        
    Returns:
        tuple: (concatenated signals, original data shape)
    """
    file_path = os.path.join(data_path, f"./DK68/BOLD_timeseries_{state}.mat")
    data = loadmat(file_path)
    data = data[f"BOLD_timeseries_{state}"]
    signals = np.hstack(data.flatten())
    print(f"{state} concatenated signals: {signals.shape} from {data.shape[0]} patients.")
    return signals, data.shape

# Set your data path
data_path = "./data"  # Modify this to your data directory

# Load data for each state
bold_awake_signals, awake_shape = load_bold_data(data_path, "Awake")
bold_deep_signals, deep_shape = load_bold_data(data_path, "Deep")
bold_recov_signals, recov_shape = load_bold_data(data_path, "Recovery")

# Quick visualization of raw signals
plt.figure(figsize=(15, 8))
plt.subplot(311)
plt.plot(bold_awake_signals[0, :1000], label='Region 1')
plt.plot(bold_awake_signals[1, :1000], label='Region 2')
plt.title('Awake State - Raw BOLD Signals')
plt.legend()

plt.subplot(312)
plt.plot(bold_deep_signals[0, :1000], label='Region 1')
plt.plot(bold_deep_signals[1, :1000], label='Region 2')
plt.title('Deep Sleep State - Raw BOLD Signals')
plt.legend()

plt.subplot(313)
plt.plot(bold_recov_signals[0, :1000], label='Region 1')
plt.plot(bold_recov_signals[1, :1000], label='Region 2')
plt.title('Recovery State - Raw BOLD Signals')
plt.legend()

plt.tight_layout()
plt.show()

## Signal Processing and Phase Computation

Now we process the BOLD signals to extract meaningful phase information:

1. **Z-score Normalization**: 
   - Standardizes the signals to have zero mean and unit variance
   - Makes signals comparable across regions and subjects

2. **Hilbert Transform**:
   - Converts real signals into analytic signals
   - Allows extraction of instantaneous phase information
   - Helps identify synchronization patterns

3. **Phase Difference Computation**:
   - Calculates phase relationships between brain regions
   - Reveals temporal coordination patterns

In [ ]:
def process_bold_signals(signals, T_shift=9):
    """Process BOLD signals to extract phase patterns.
    
    Args:
        signals (np.ndarray): BOLD time series data
        T_shift (int): Time shift for analysis window
        
    Returns:
        tuple: (phase patterns, synchronization data)
    """
    # Z-score normalization
    signals_zscore = stats.zscore(signals, axis=1)
    
    # Compute analytic signal using Hilbert transform
    analytic_signal = scipy.signal.hilbert(signals_zscore)
    phases = np.angle(analytic_signal)
    
    # Initialize arrays
    N, L = signals.shape
    T = np.arange(T_shift, L-T_shift)
    pattern = np.zeros((len(T), int(N*(N-1)/2)))
    sync_data = np.zeros(len(T))
    
    # Compute phase patterns
    for t_idx, t in enumerate(T):
        # Global synchronization
        complex_order = np.mean(np.exp(1j * phases[:, t]))
        sync_data[t_idx] = np.abs(complex_order)
        
        # Pairwise phase relationships
        phase_diff = np.zeros((N, N))
        for i in range(N):
            for j in range(i):
                phase_diff[i,j] = np.cos(np.angle(np.exp(1j*(phases[i,t] - phases[j,t]))))
        
        pattern[t_idx] = squareform(phase_diff, checks=False)
    
    return pattern, sync_data

# Process each state
print("Processing Awake state...")
pattern_awake, sync_awake = process_bold_signals(bold_awake_signals)
print("Processing Deep Sleep state...")
pattern_deep, sync_deep = process_bold_signals(bold_deep_signals)
print("Processing Recovery state...")
pattern_recov, sync_recov = process_bold_signals(bold_recov_signals)

# Visualize global synchronization
plt.figure(figsize=(12, 6))
plt.plot(sync_awake, label='Awake', alpha=0.7)
plt.plot(sync_deep, label='Deep Sleep', alpha=0.7)
plt.plot(sync_recov, label='Recovery', alpha=0.7)
plt.title('Global Synchronization Over Time')
plt.xlabel('Time Point')
plt.ylabel('Synchronization Strength')
plt.legend()
plt.show()

## K-means Clustering Analysis

We use k-means clustering to identify distinct phase patterns (brain states) in the data. This process involves:

1. **Data Cleaning**:
   - Remove empty patterns (all zeros)
   - Remove outlier patterns using cityblock distance
   
2. **Clustering**:
   - Apply k-means to find recurring phase patterns
   - Use cityblock (Manhattan) distance as similarity metric
   - Identify cluster centers representing distinct brain states

3. **Analysis**:
   - Compare brain states across consciousness conditions
   - Calculate state probabilities and transitions
   - Examine temporal dynamics

In [ ]:
def cluster_phase_patterns(pattern, n_clusters=5):
    """Perform k-means clustering on phase patterns.
    
    Args:
        pattern (np.ndarray): Phase pattern data
        n_clusters (int): Number of clusters
        
    Returns:
        tuple: (cluster indices, cluster centers, cleaned pattern data)
    """
    # Remove empty patterns
    good_pattern = np.sum(np.abs(pattern), axis=1) > 0
    pattern_clean = pattern[good_pattern]
    print(f"Removed {np.sum(~good_pattern)} empty patterns")
    
    # Remove outliers using cityblock distance
    D = np.zeros((pattern_clean.shape[0], pattern_clean.shape[0]))
    for i in range(pattern_clean.shape[0]):
        for j in range(i):
            D[i,j] = D[j,i] = cityblock(pattern_clean[i], pattern_clean[j])
    
    D_mean = np.mean(D, axis=0)
    D_zscore = stats.zscore(D_mean)
    good_pattern = D_zscore < 3
    pattern_clean = pattern_clean[good_pattern]
    print(f"Removed {np.sum(~good_pattern)} outlier patterns")
    
    # Perform k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_idx = kmeans.fit_predict(pattern_clean)
    
    return cluster_idx, kmeans.cluster_centers_, pattern_clean

# Cluster each state
n_clusters = 5
print("\nClustering Awake state patterns...")
idx_awake, centers_awake, clean_awake = cluster_phase_patterns(pattern_awake, n_clusters)
print("\nClustering Deep Sleep state patterns...")
idx_deep, centers_deep, clean_deep = cluster_phase_patterns(pattern_deep, n_clusters)
print("\nClustering Recovery state patterns...")
idx_recov, centers_recov, clean_recov = cluster_phase_patterns(pattern_recov, n_clusters)

## Visualization of Results

We create several visualizations to analyze and compare the brain states across conditions:

1. **Brain State Patterns**:
   - Visualization of cluster centers
   - Shows typical phase relationships for each state
   
2. **State Probabilities**:
   - Distribution of brain states in each condition
   - Reveals changes in brain dynamics across conditions
   
3. **Temporal Dynamics**:
   - State transitions over time
   - Temporal stability of brain states

In [ ]:
def plot_brain_states(centers, title):
    """Plot brain state patterns identified by clustering.
    
    Args:
        centers (np.ndarray): Cluster centers
        title (str): Plot title
    """
    n_states = centers.shape[0]
    fig, axes = plt.subplots(1, n_states, figsize=(4*n_states, 4))
    
    for i in range(n_states):
        im = axes[i].imshow(squareform(centers[i]), cmap='RdBu_r', vmin=-1, vmax=1)
        axes[i].set_title(f'State {i+1}')
        plt.colorbar(im, ax=axes[i])
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# Plot brain states for each condition
print("Brain States in Awake Condition:")
plot_brain_states(centers_awake, 'Awake Brain States')
print("\nBrain States in Deep Sleep Condition:")
plot_brain_states(centers_deep, 'Deep Sleep Brain States')
print("\nBrain States in Recovery Condition:")
plot_brain_states(centers_recov, 'Recovery Brain States')

In [ ]:
# Plot state probabilities
def plot_state_probabilities(idx_list, labels):
    """Plot probability distribution of brain states.
    
    Args:
        idx_list (list): List of cluster indices for each condition
        labels (list): Condition labels
    """
    probs = [np.bincount(idx, minlength=n_clusters) / len(idx) for idx in idx_list]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(n_clusters)
    width = 0.25
    
    for i, (prob, label) in enumerate(zip(probs, labels)):
        ax.bar(x + i*width, prob, width, label=label)
    
    ax.set_xlabel('Brain State')
    ax.set_ylabel('Probability')
    ax.set_title('Brain State Probability Distribution')
    ax.set_xticks(x + width)
    ax.set_xticklabels([f'State {i+1}' for i in range(n_clusters)])
    ax.legend()
    
    plt.tight_layout()
    plt.show()

# Plot state probabilities
plot_state_probabilities(
    [idx_awake, idx_deep, idx_recov],
    ['Awake', 'Deep Sleep', 'Recovery']
)

In [ ]:
# Analyze temporal dynamics
def plot_state_transitions(idx, title):
    """Plot state transitions over time.
    
    Args:
        idx (np.ndarray): Cluster indices
        title (str): Plot title
    """
    # Calculate transition matrix
    trans_mat = np.zeros((n_clusters, n_clusters))
    for i in range(len(idx)-1):
        trans_mat[idx[i], idx[i+1]] += 1
    
    # Normalize
    row_sums = trans_mat.sum(axis=1)
    trans_mat = trans_mat / row_sums[:, np.newaxis]
    
    # Plot
    plt.figure(figsize=(8, 6))
    sns.heatmap(trans_mat, annot=True, fmt='.2f', cmap='YlOrRd')
    plt.title(f'State Transition Probabilities\n{title}')
    plt.xlabel('To State')
    plt.ylabel('From State')
    plt.show()

# Plot transition matrices
print("State Transitions in Awake Condition:")
plot_state_transitions(idx_awake, 'Awake')
print("\nState Transitions in Deep Sleep Condition:")
plot_state_transitions(idx_deep, 'Deep Sleep')
print("\nState Transitions in Recovery Condition:")
plot_state_transitions(idx_recov, 'Recovery')

## Conclusions

The analysis reveals several key findings:

1. **Brain State Patterns**:
   - Each consciousness state shows distinct phase synchronization patterns
   - Some brain states are shared across conditions, while others are condition-specific

2. **State Probabilities**:
   - The distribution of brain states varies between consciousness conditions
   - Certain states are more prevalent in specific conditions

3. **Temporal Dynamics**:
   - State transitions show different patterns across conditions
   - Some states are more stable (self-transitions) in certain conditions

These results provide insights into how brain dynamics change across different states of consciousness, potentially helping us understand the neural correlates of consciousness.